# site

> The `Site` class — geometry + identity for polygon-first analysis.

In [1]:
#| default_exp site

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from __future__ import annotations
import json
from pathlib import Path
from datetime import datetime
from typing import Optional, Union
from fastcore.basics import patch

import ee
import geemap
from pyproj import Transformer

/Users/alice/.local/share/mamba/envs/gee-polygons/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## Site

The `Site` class is the fundamental unit of analysis in gee-polygons. It wraps a polygon geometry with temporal metadata, making it easy to extract time-series features from Google Earth Engine.

A Site represents a single spatial unit (e.g., a restoration plot, conservation area, or agricultural field) that you want to analyze over time.

In [4]:
#| export
class Site:
    """A polygon with temporal context for GEE analysis.
    
    The Site class wraps an ee.Feature with convenient accessors for
    common properties like start_year and area, plus helper methods
    for temporal analysis.
    """
    
    def __init__(self, 
                 feature: ee.Feature,
                 start_year: Optional[int] = None,
                 site_id: Optional[str] = None):
        """Create a Site from an ee.Feature.
        
        Args:
            feature: An ee.Feature with polygon geometry
            start_year: Override start year (otherwise reads from feature properties)
            site_id: Override site ID (otherwise reads from 'rid' property)
        """
        self._feature = feature
        self._start_year = start_year
        self._site_id = site_id
    
    @property
    def feature(self) -> ee.Feature:
        """The underlying ee.Feature."""
        return self._feature
    
    @property
    def geometry(self) -> ee.Geometry:
        """The site's geometry."""
        return self._feature.geometry()
    
    @property
    def start_year(self) -> int:
        """The year this site's monitoring/restoration began."""
        if self._start_year is not None:
            return self._start_year
        # Try to get from feature properties
        return int(self._feature.get('start_year').getInfo())
    
    @property
    def site_id(self) -> str:
        """Unique identifier for this site."""
        if self._site_id is not None:
            return self._site_id
        return str(self._feature.get('rid').getInfo())
    
    @property
    def area_ha(self) -> float:
        """Site area in hectares."""
        return float(self._feature.get('area_ha').getInfo())
    
    @property 
    def properties(self) -> dict:
        """All feature properties as a dictionary."""
        return self._feature.getInfo()['properties']
    
    def buffer(self, distance_m: float) -> ee.Geometry:
        """Buffer the site geometry by a distance in meters.
        
        Args:
            distance_m: Buffer distance in meters
            
        Returns:
            Buffered ee.Geometry
        """
        return self.geometry.buffer(distance_m)
    
    def years(self, end_year: Optional[int] = None) -> list[int]:
        """Generate list of years from start_year to end_year.
        
        Args:
            end_year: End year (defaults to current year)
            
        Returns:
            List of years for time-series analysis
        """
        if end_year is None:
            end_year = datetime.now().year
        return list(range(self.start_year, end_year + 1))
    
    def __repr__(self) -> str:
        return f"Site(id={self.site_id}, start_year={self.start_year})"
    
    @classmethod
    def from_geojson(cls, 
                     feature_dict: dict,
                     source_crs: str = 'EPSG:4326') -> Site:
        """Create a Site from a GeoJSON feature dictionary.
        
        Args:
            feature_dict: A GeoJSON Feature dict with geometry and properties
            source_crs: The CRS of the input coordinates (default WGS84)
            
        Returns:
            A Site instance
        """
        geom = feature_dict['geometry']
        props = feature_dict.get('properties', {})
        
        # Reproject coordinates if not WGS84
        if source_crs != 'EPSG:4326':
            geom = _reproject_geometry(geom, source_crs, 'EPSG:4326')
        
        # Create ee.Geometry based on type
        geom_type = geom['type']
        coords = geom['coordinates']
        
        if geom_type == 'Polygon':
            ee_geom = ee.Geometry.Polygon(coords)
        elif geom_type == 'MultiPolygon':
            ee_geom = ee.Geometry.MultiPolygon(coords)
        elif geom_type == 'Point':
            ee_geom = ee.Geometry.Point(coords)
        else:
            raise ValueError(f"Unsupported geometry type: {geom_type}")
        
        # Create ee.Feature with properties
        ee_feature = ee.Feature(ee_geom, props)
        
        return cls(ee_feature)

In [5]:
#| export
def _reproject_geometry(geom: dict, 
                        source_crs: str, 
                        target_crs: str) -> dict:
    """Reproject a GeoJSON geometry from source to target CRS.
    
    Args:
        geom: GeoJSON geometry dict
        source_crs: Source CRS (e.g., 'EPSG:5880')
        target_crs: Target CRS (e.g., 'EPSG:4326')
        
    Returns:
        Reprojected geometry dict (2D only, Z coordinates stripped)
    """
    transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)
    
    def transform_coords(coords):
        """Recursively transform coordinates."""
        if isinstance(coords[0], (int, float)):
            # This is a coordinate pair - transform and return 2D only
            x, y = transformer.transform(coords[0], coords[1])
            return [x, y]  # Strip Z coordinate for GEE compatibility
        else:
            # This is a list of coordinates/rings
            return [transform_coords(c) for c in coords]
    
    return {
        'type': geom['type'],
        'coordinates': transform_coords(geom['coordinates'])
    }

## Loading Sites

Helper functions to load sites from common formats.

In [6]:
#| export
def load_sites(path: Union[str, Path], 
               source_crs: Optional[str] = None) -> list[Site]:
    """Load sites from a GeoJSON file.
    
    Args:
        path: Path to GeoJSON file
        source_crs: Override source CRS (auto-detected from file if present)
        
    Returns:
        List of Site objects
    """
    path = Path(path)
    
    with open(path) as f:
        data = json.load(f)
    
    # Try to detect CRS from file
    if source_crs is None:
        source_crs = _detect_crs(data)
    
    features = data.get('features', [data])  # Handle single feature or collection
    
    return [Site.from_geojson(f, source_crs=source_crs) for f in features]


def _detect_crs(geojson_data: dict) -> str:
    """Detect CRS from GeoJSON data, defaulting to WGS84."""
    crs_info = geojson_data.get('crs', {})
    crs_props = crs_info.get('properties', {})
    crs_name = crs_props.get('name', '')
    
    # Parse common CRS formats
    if 'EPSG' in crs_name:
        # Extract EPSG code from URN or direct format
        # e.g., "urn:ogc:def:crs:EPSG::5880" -> "EPSG:5880"
        parts = crs_name.split(':')
        for i, part in enumerate(parts):
            if part == 'EPSG' and i + 1 < len(parts):
                code = parts[-1]  # Get the last part (the actual code)
                if code:  # Handle double colon case
                    return f'EPSG:{code}'
    
    # Default to WGS84
    return 'EPSG:4326'

## Example Usage

Let's load some restoration sites and explore a single site.

In [7]:
# Initialize Earth Engine (uncomment when running)
ee.Authenticate()
ee.Initialize(project="hs-brazilreforestation")

In [8]:
# Load sites from GeoJSON
sites = load_sites('../data/restoration_sites_subset.geojson')
print(f"Loaded {len(sites)} sites")

Loaded 10 sites


In [9]:
# Explore a single site
site = sites[0]
print(site)
print(f"Start year: {site.start_year}")
print(f"Area: {site.area_ha:.2f} ha")
print(f"Years for analysis: {site.years(end_year=2024)}")

Site(id=3107, start_year=2016)
Start year: 2016
Area: 37.09 ha
Years for analysis: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [10]:
# Get buffered geometry for context
buffer_1km = site.buffer(1000)
print(f"Buffered geometry created")

Buffered geometry created


## Visualizing Sites

Now that we have a site loaded, we'd like to see it on a map. Let's add a `show()` method using geemap's interactive mapping.

In [11]:
#| export
@patch
def show(self: Site, 
         zoom: int = 14,
         basemap: str = 'SATELLITE',
         color: str = 'blue',
         fill_color: str = '#0000ff33') -> geemap.Map:
    """Display the site on an interactive map.
    
    Args:
        zoom: Initial zoom level (default 14)
        basemap: Basemap type - 'SATELLITE', 'ROADMAP', 'TERRAIN', 'HYBRID'
        color: Outline color (default 'blue')
        fill_color: Fill color with alpha (default semi-transparent blue)
        
    Returns:
        A geemap.Map object centered on the site
    """
    m = geemap.Map()
    m.add_basemap(basemap)
    m.center_object(self.geometry, zoom)
    
    # Style the site polygon
    style = {'color': color, 'fillColor': fill_color, 'width': 2}
    m.add_layer(self.geometry, style, f'Site {self.site_id}')
    
    return m

In [12]:
# View the site on a satellite map
site.show()

Map(center=[-6.500137640908457, -51.08157294649402], controls=(WidgetControl(options=['position', 'transparent…

In [13]:
#| hide
import nbdev; nbdev.nbdev_export()